In [2]:
import ivolatility as ivol
import dotenv
import os
import pandas as pd

dotenv.load_dotenv()
api_key = os.getenv("IVOL_API_KEY")
if not api_key:
    raise RuntimeError("❌ IVOL_API_KEY not found in .env file or environment.")

ivol.setLoginParams(apiKey=api_key)
api_key


'BafOt06jgw7YLpN2'

In [5]:

getMarketData = ivol.setMethod('/equities/eod/ivs')
marketData = getMarketData(symbol='AAPL', date='2021-12-16')
marketData = getMarketData(symbol='AAPL', from_='2021-12-10', to='2021-12-17')
marketData


KeyboardInterrupt: 

In [6]:
getMarketData = ivol.setMethod('/equities/eod/option-series-on-date')
marketData = getMarketData(symbol='AAPL', expFrom='2022-10-21', expTo='2022-10-21', strikeFrom=100, strikeTo=105, callPut='C', date = '2022-02-16')
marketData

,OptionSymbol,callPut,strike,expirationDate,optionId
0,AAPL 221021C00100000,C,100.0,2022-10-21,118633947
1,AAPL 221021C00105000,C,105.0,2022-10-21,118781541


In [13]:
#user says i want to price a oct 2025 200 300 collar for aapl
getMarketData = ivol.setMethod('/equities/option-series')

#compute first date for the month
dt_start = pd.to_datetime('2026-10-01')
dt_end = pd.to_datetime('2026-10-31')

output={}
output["issues"]=[]

marketData = getMarketData(symbol='AAPL', expFrom=dt_start.strftime('%Y-%m-%d'), expTo=dt_end.strftime('%Y-%m-%d'), strikeFrom=200, strikeTo=300, callPut='C')
#get all the individual maturities
if (marketData.empty):
    output["issues"].append("No data found")
else:
    maturities = marketData['expirationDate'].unique()
    if (len(maturities) >1):
        output["issues"].append("Multiple maturities found")
        output["possible_maturities"]=maturities
    else:
        maturity = maturities[0]

#choose one maturity
maturity = maturities[0]

#get the option series for the chosen maturity
option_series = marketData[marketData['expirationDate'] == maturity]


#check if the strike exists otherwise suggest nearest strikes above and below
if 200 not in option_series['strike'].values:
    output["issues"].append("Strike 200 not found")
    strike_low = option_series[option_series['strike'] < 200].iloc[-1]['strike']
    strike_low = option_series[option_series['strike'] > 200].iloc[0]['strike']
else:
    strike_low = 200
   


if 300 not in option_series['strike'].values:
    output["issues"].append("Strike 300 not found")
    strike_high = option_series[option_series['strike'] < 300].iloc[-1]['strike']
    strike_high = option_series[option_series['strike'] > 300].iloc[0]['strike']
else:
    strike_high = 300













NameError: name 'maturities' is not defined

In [4]:
#user says i want to price a oct 2025 200 300 collar for aapl
getMarketData = ivol.setMethod('/equities/option-series')
marketData = getMarketData(symbol='AAPL')#, strikeFrom=200, strikeTo=300, callPut='C')
#get all the individual maturities
marketData

,OptionSymbol,callPut,strike,expirationDate,optionId
0,AAPL 250926C00110000,C,110.0,2025-09-26,140370151
1,AAPL 250926P00110000,P,110.0,2025-09-26,140370152
2,AAPL 250926C00120000,C,120.0,2025-09-26,140370153
3,AAPL 250926P00120000,P,120.0,2025-09-26,140370154
4,AAPL 250926C00125000,C,125.0,2025-09-26,140370155
...,...,...,...,...,...
2439,AAPL 280121P00430000,P,430.0,2028-01-21,141281966
2440,AAPL 280121C00440000,C,440.0,2028-01-21,141281967
2441,AAPL 280121P00440000,P,440.0,2028-01-21,141281968
2442,AAPL 280121C00450000,C,450.0,2028-01-21,141281969


In [88]:
from datetime import date, timedelta
import re

def third_friday(year: int, month: int) -> date:
    """3rd Friday of (year, month)."""
    d = date(year, month, 15)  # 15th is always on/after 2nd Friday
    return d + timedelta(days=(4 - d.weekday()) % 7)  # 0=Mon..4=Fri

def pick_expiry(series, year: int, month: int) -> str | None:
    """
    Choose one expiry:
      1) Prefer expiries in the requested month.
      2) Minimize distance to that month’s 3rd Friday.
      3) Tie-breaker: earlier date wins.
    Returns ISO date string 'YYYY-MM-DD' or None.
    """
    print(f"calling pick_expiry, month: {month}, year: {year}")
    records = series if isinstance(series, list) else series.to_dict("records")
    exps = []
    for r in records:
        s = r.get("expiration_date") or r.get("expirationDate")
        if not s: 
            continue
        y, m, d = map(int, s.split("-"))
        exps.append(date(y, m, d))
    if not exps:
        return None

    target = third_friday(year, month)
    print(f"target: {target}")
    in_month = [e for e in set(exps) if e.year == year and e.month == month]
    candidates = in_month or list(set(exps))
    chosen = min(candidates, key=lambda e: (abs((e - target).days), e))
    return chosen.isoformat()

def pick_strike(series, expiration_iso: str, cp: str, target_k: float):
    """
    From (series, expiration, cp in {'P','C'}), pick nearest listed strike.
    Tie-breaker: lower strike wins when distances are equal.
    Returns (matched, nearest_below, nearest_above, exact_match: bool) or (None, None, None, False) if no strikes.
    """
    cp = cp.upper()[0]
    records = series if isinstance(series, list) else series.to_dict("records")

    def row_cp(r):
        return (r.get("cp") or r.get("call_put") or r.get("optionType") or "").upper()[:1]
    def row_exp(r):
        return r.get("expiration_date") or r.get("expirationDate")

    strikes = sorted({
        float(r["price_strike"]) for r in records
        if row_exp(r) == expiration_iso and row_cp(r) == cp and r.get("price_strike") is not None
    })
    if not strikes:
        return None, None, None, False

    matched = min(strikes, key=lambda s: (abs(s - target_k), s))
    below = max([s for s in strikes if s <= target_k], default=strikes[0])
    above = min([s for s in strikes if s >= target_k], default=strikes[-1])
    return matched, below, above, (matched == float(target_k))



_MONTHS = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,
           'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}

def parse_expiry_mmmyy(expiry_str: str, *, pivot: int = 69) -> tuple[int, int]:
    """
    Parse 'MMMYY' or 'MMMYYYY' -> (month, year).

    - Month: 3-letter English month (case-insensitive).
    - Year: 2 or 4 digits.
      For 2-digit years: <= pivot -> 2000+YY, else -> 1900+YY.
      (Default pivot=69: '26'->2026, '99'->1999.)
    - Accepts optional separators between month and year (space, '-', '/', '_').

    Raises ValueError on invalid input.
    """
    m = re.fullmatch(r'\s*([A-Za-z]{3})[ \-_/]*([0-9]{2}|[0-9]{4})\s*', expiry_str)
    if not m:
        raise ValueError(f"Invalid expiry '{expiry_str}'. Use formats like 'Jan26' or 'Jan2026'.")

    mon = m.group(1).lower()
    if mon not in _MONTHS:
        raise ValueError(f"Invalid month '{mon}'. Use 3-letter English month (e.g., Jan, Feb).")

    year_part = m.group(2)
    if len(year_part) == 2:
        yy = int(year_part)
        year = 2000 + yy if yy <= pivot else 1900 + yy
    else:
        year = int(year_part)

    return _MONTHS[mon], year

# Examples
expiry_month, expiry_year = parse_expiry_mmmyy("jan26")     # (1, 2026)
# parse_expiry_mmmyy("Oct2025")  -> (10, 2025)
# parse_expiry_mmmyy("DEC-99")   -> (12, 1999)

In [ ]:
expiry=pick_expiry(marketData,2026,1)
(matched, below, above, exact_match) =pick_strike(marketData, expiry, 'P', 200)
(matched, below, above, exact_match) =pick_strike(marketData, expiry, 'C', 300)
option1_row=marketData[marketData['strike']==matched]
option1_row=option1_row[option1_row['']=='P']
option1_row=option1_row[option1_row['expirationDate']==expiry]

option2_row=marketData[marketData['strike']==matched]
option2_row=option2_row[option2_row['callPut']=='C']
option2_row=option2_row[option2_row['expirationDate']==expiry]
option2_row["optionId"]





KeyError: 'strike'

In [85]:
expiry_str="jan26"
expiry_month=1
expiry_year=2026

legs=[{"cp":"P","strike":250,"expiry_month":expiry_month,"expiry_year":expiry_year},
      {"cp":"C","strike":300,"expiry_month":expiry_month,"expiry_year":expiry_year}]
legs





[{'cp': 'P', 'strike': 250, 'expiry_month': 1, 'expiry_year': 2026},
 {'cp': 'C', 'strike': 300, 'expiry_month': 1, 'expiry_year': 2026}]

In [86]:
getMarketData = ivol.setMethod('/equities/eod/stock-opts-by-param')
marketData = getMarketData(symbol='AAPL',tradeDate='2025-09-25',dteFrom=0,dteTo=760,moneynessFrom=-100,moneynessTo=100,cp='C')
marketData_puts=getMarketData(symbol='AAPL',tradeDate='2025-09-25',dteFrom=0,dteTo=760,moneynessFrom=-100,moneynessTo=100,cp='P')
marketData=pd.concat([marketData,marketData_puts])
rows=[]
for option in legs:
    expiry=pick_expiry(marketData,option["expiry_year"],option["expiry_month"])
    print(f"expiry: {expiry}")
    (matched, below, above, exact_match) =pick_strike(marketData, expiry, option["cp"], option["strike"])
    option_row=marketData[(marketData['price_strike']==matched) & (marketData['expiration_date']==expiry) & (marketData['call_put']==option["cp"])]
    rows.append(option_row)
df_out=pd.concat(rows)
df_out2=df_out[['expiration_date','price_strike','call_put','Bid','Ask',"iv","delta",'gamma','theta','vega',"rho",'iv',"volume","openinterest","underlying_price"]]


calling pick_expiry, month: 1, year: 2026
target: 2026-01-16
expiry: 2026-01-16
calling pick_expiry, month: 1, year: 2026
target: 2026-01-16
expiry: 2026-01-16


In [87]:
df_out2

,expiration_date,price_strike,call_put,Bid,Ask,iv,delta,gamma,theta,vega,rho,iv,volume,openinterest,underlying_price
532,2026-01-16,250.0,P,9.65,9.80,0.247776,-0.373255,0.011067,-0.049769,0.542284,-0.260468,0.247776,188,4799,256.87
542,2026-01-16,300.0,C,2.26,2.33,0.229673,0.144163,0.006922,-0.036724,0.324221,0.107463,0.229673,2860,47021,256.87


In [89]:
import pandas as pd

def _sorted_unique_expiries(df: pd.DataFrame) -> list[str]:
    exps = pd.to_datetime(df["expiration_date"]).dt.date.astype(str).unique().tolist()
    return sorted(exps)  # ISO yyyy-mm-dd sorts chronologically

def _resolve_leg_expiry(df: pd.DataFrame, leg: dict, default_year: int | None, default_month: int | None) -> tuple[str | None, list]:
    """
    Return (expiry_iso, issues[]) for a single leg using one of:
      - expiry_str ('Jan26', 'Oct2025')
      - expiry_month + expiry_year
      - exp_rank (0=front, 1=back, ...)
      - exp_hint ('front'|'back')
      Else: falls back to default (month/year) via pick_expiry, or front if no default.
    """
    issues = []
    # 1) explicit month/year via string
    if "expiry_str" in leg and leg["expiry_str"]:
        try:
            m, y = parse_expiry_mmmyy(str(leg["expiry_str"]))
            return pick_expiry(df, y, m), issues
        except Exception as e:
            issues.append({"code":"BAD_EXPIRY_FORMAT","msg":f"Could not parse expiry_str '{leg['expiry_str']}': {e}"})
            return None, issues

    # 2) explicit month/year via ints
    if leg.get("expiry_month") and leg.get("expiry_year"):
        return pick_expiry(df, int(leg["expiry_year"]), int(leg["expiry_month"])), issues

    # 3) rank/hint based (calendar style)
    rank = None
    if "exp_rank" in leg and leg["exp_rank"] is not None:
        rank = int(leg["exp_rank"])
    elif leg.get("exp_hint"):
        hint = str(leg["exp_hint"]).lower()
        rank = 0 if hint == "front" else (1 if hint == "back" else None)

    if rank is not None:
        exps = _sorted_unique_expiries(df)
        if rank < len(exps):
            return exps[rank], issues
        issues.append({"code":"EXPIRY_RANK_OOB","msg":f"Requested exp_rank={rank}, but only {len(exps)} expiries available."})
        return None, issues

    # 4) fallback: default month/year if provided, else front
    if default_year and default_month:
        return pick_expiry(df, int(default_year), int(default_month)), issues

    exps = _sorted_unique_expiries(df)
    return (exps[0] if exps else None), issues

def resolve_option_rows_multi(
    marketData: pd.DataFrame,
    symbol: str,
    legs: list[dict],
    default_year: int | None = None,
    default_month: int | None = None,
    strike_tol: float = 1e-6
):
    """
    legs examples:
      # calendar: same strike, different expiries
      [{"cp":"C","strike":100,"exp_hint":"front"},
       {"cp":"C","strike":100,"exp_hint":"back"}]

      # diagonal: different strikes & explicit months
      [{"cp":"P","strike":250,"expiry_str":"Jan26"},
       {"cp":"P","strike":240,"expiry_str":"Mar26"}]

      # single-month default with one override
      [{"cp":"P","strike":200}, {"cp":"C","strike":300,"exp_rank":1}]

    Returns: (result_dict, df_out)
    """
    issues = []
    df = marketData.copy()
    # normalize columns
    df["call_put"] = df["call_put"].str.upper().str[0]
    df["expiration_date"] = pd.to_datetime(df["expiration_date"]).dt.date.astype(str)
    df["price_strike"] = pd.to_numeric(df["price_strike"], errors="coerce")
    df = df.dropna(subset=["expiration_date","call_put","price_strike"])

    rows = []
    norm_legs = []

    # resolve each leg independently (expiry may differ)
    for i, leg in enumerate(legs):
        cp = leg["cp"].upper()[0]
        k  = float(leg["strike"])

        expiry, leg_issues = _resolve_leg_expiry(df, leg, default_year, default_month)
        issues.extend(leg_issues)
        if not expiry:
            return (
                {"ok": False, "dataset": [], "normalized": None,
                 "issues": issues + [{"code":"NO_EXPIRY","msg":f"No expiry could be resolved for leg {i}."}]},
                pd.DataFrame()
            )

        matched, below, above, exact = pick_strike(df, expiry, cp, k)
        if matched is None:
            return (
                {"ok": False, "dataset": [], "normalized": None,
                 "issues": issues + [{"code":"LEG_MISSING","msg":f"No {cp} strikes near {k} @ {expiry}."}]},
                pd.DataFrame()
            )
        if not exact:
            issues.append({"code":"STRIKE_ADJUSTED",
                           "msg": f"{int(k)} {cp} not listed @ {expiry}; used {matched} (below {below}, above {above})."})

        sel = df[
            (df["expiration_date"] == expiry) &
            (df["call_put"] == cp) &
            (df["price_strike"].sub(matched).abs() <= strike_tol)
        ]
        if sel.empty:
            return (
                {"ok": False, "dataset": [], "normalized": None,
                 "issues": issues + [{"code":"LEG_MISSING","msg":f"Matched {cp} {matched} not found @ {expiry}."}]},
                pd.DataFrame()
            )

        # prefer non-settlement, then highest OI, then volume
        if "is_settlement" not in sel.columns: sel["is_settlement"] = 0
        if "openinterest" not in sel.columns: sel["openinterest"] = 0
        if "volume" not in sel.columns: sel["volume"] = 0

        best = sel.sort_values(by=["is_settlement","openinterest","volume"], ascending=[True, False, False]).iloc[0]
        rows.append(best)
        norm_legs.append({"cp": cp, "requested": k, "matched": float(matched), "expiration": expiry})

    # Optional: detect calendars that accidentally landed on same expiry
    exps_used = {r["expiration_date"] for r in rows}
    if len(exps_used) != len(rows):
        issues.append({"code":"EXPIRIES_NOT_DISTINCT","msg":"Multiple legs resolved to the same expiry."})

    # build compact output table + dataset
    keep = ["option_id","option_symbol","expiration_date","call_put","price_strike",
            "Bid","Ask","iv","delta","gamma","theta","vega","rho","volume","openinterest","underlying_price","dte","is_settlement"]
    keep = [c for c in keep if c in df.columns]
    df_out = pd.DataFrame(rows)[keep].reset_index(drop=True)

    dataset = [{
        "optionId": row.get("option_id") or row.get("option_symbol"),
        "symbol": symbol.upper(),
        "cp": row["call_put"],
        "expiration": row["expiration_date"],
        "strike": float(row["price_strike"]),
        "osym": row.get("option_symbol")
    } for _, row in df_out.iterrows()]

    result = {
        "ok": True,
        "dataset": dataset,
        "normalized": {"symbol": symbol.upper(), "legs": norm_legs},
        "issues": issues
    }
    return result, df_out

In [92]:
result,df_out=resolve_option_rows_multi(marketData,"AAPL",legs)
df_out

calling pick_expiry, month: 1, year: 2026
target: 2026-01-16
calling pick_expiry, month: 1, year: 2026
target: 2026-01-16


,option_id,option_symbol,expiration_date,call_put,price_strike,Bid,Ask,iv,delta,gamma,theta,vega,rho,volume,openinterest,underlying_price,dte,is_settlement
0,128416790,AAPL 260116P00250000,2026-01-16,P,250.0,9.65,9.80,0.247776,-0.373255,0.011067,-0.049769,0.542284,-0.260468,188,4799,256.87,113,0
1,128416807,AAPL 260116C00300000,2026-01-16,C,300.0,2.26,2.33,0.229673,0.144163,0.006922,-0.036724,0.324221,0.107463,2860,47021,256.87,113,0


In [73]:
marketData.columns

Index(['c_date', 'option_symbol', 'dte', 'stocks_id', 'expiration_date',
       'call_put', 'price_strike', 'price_open', 'price_high', 'price_low',
       'price', 'volume', 'openinterest', 'iv', 'delta', 'preiv', 'gamma',
       'theta', 'vega', 'rho', 'Ask', 'Bid', 'underlying_price', 'calc_OTM',
       'option_id', 'is_settlement'],
      dtype='object')

In [93]:
getMarketData = ivol.setMethod('/curves/eod/zero-curve')
marketData = getMarketData(date='2025-09-25', ticker='AAPL')
marketData


HTTPError: 403 Client Error: Forbidden for url: https://restapi.ivolatility.com/curves/eod/zero-curve?date=2025-09-25&ticker=AAPL

In [66]:
marketData = getMarketData(symbol='AAPL',tradeDate='2025-09-25',dteFrom=0,dteTo=760,moneynessFrom=-100,moneynessTo=100,cp='C')
marketData["price_strike"].unique()

array([110. , 120. , 125. , 130. , 135. , 140. , 145. , 150. , 155. ,
       160. , 165. , 170. , 175. , 180. , 185. , 190. , 195. , 200. ,
       202.5, 205. , 207.5, 210. , 212.5, 215. , 217.5, 220. , 222.5,
       225. , 227.5, 230. , 232.5, 235. , 237.5, 240. , 242.5, 245. ,
       247.5, 250. , 252.5, 255. , 257.5, 260. , 262.5, 265. , 267.5,
       270. , 272.5, 275. , 277.5, 280. , 282.5, 285. , 287.5, 290. ,
       292.5, 295. , 300. , 305. , 310. , 315. , 320. , 325. ,  90. ,
        95. , 100. , 105. , 115. , 330. , 335. , 340. , 345. , 350. ,
       355. , 360. , 370. ,   5. ,  10. ,  15. ,  20. ,  25. ,  30. ,
        35. ,  40. ,  45. ,  50. ,  55. ,  60. ,  65. ,  70. ,  75. ,
        80. ,  85. , 380. , 390. , 400. , 410. , 420. , 430. , 440. ,
       450. ])

In [ ]:
getMarketData = ivol.setMethod('/equities/option-series')
marketData = getMarketData(symbol='AAPL', expFrom='2024-10-01', expTo='2026-10-30', strikeFrom=100, strikeTo=300, callPut='C')
(matched, below, above, exact_match) =pick_strike(marketData, expiry, 'P', 200)
(matched, below, above, exact_match) =pick_strike(marketData, expiry, 'C', 300)



300.0

In [32]:
getMarketData = ivol.setMethod('/equities/eod/options-rawiv')
#marketData = getMarketData(symbol='AAPL', date='2021-12-16')
marketData = getMarketData(symbol='AAPL', from_='2021-12-10', to='2021-12-17')

HTTPError: 403 Client Error: Forbidden for url: https://restapi.ivolatility.com/equities/eod/options-rawiv?symbol=AAPL&to=2021-12-17&from=2021-12-10

In [31]:
getMarketData = ivol.setMethod('/equities/eod/summary')
marketData = getMarketData(symbols='AAPL, META')
marketData

,hvp150,ivx60Prev,dividendDate,change1W_percent,beta180D,ivr180,endDate,ivxlow150,companyName,openInterestCall,...,ivxLowDate60,hv180,regionId,pe,ivp180,bid,ivr30,ivx180,hv20,ivxlowdate180
0,85.599998,36.91,2025-09-22T00:00:00,-1.94,1.439606,24.344978,2079-06-06T00:00:00,0.306984,Meta Platforms Inc,1159352,...,2025-08-15,39.37,1,26.868950,35.599998,760.66,11.004140,34.70,17.88,2025-07-31
1,NaN,36.55,2025-09-22T00:00:00,-2.03,0.000000,25.764206,2079-06-06T00:00:00,0.305441,Meta Platforms Inc,619,...,2025-08-14,NaN,4,NaN,45.454544,41.27,9.563416,34.93,17.50,2025-07-31
2,90.000000,25.22,2025-08-11T00:00:00,5.41,0.000000,9.495436,2079-06-06T00:00:00,0.233324,Apple Inc,635,...,2025-09-05,NaN,4,NaN,21.678322,36.20,4.265975,25.44,28.60,2025-09-08
3,94.000000,25.73,2025-08-11T00:00:00,5.57,1.363112,15.915346,2079-06-06T00:00:00,0.213053,Apple Inc,3407823,...,2024-11-26,36.33,1,38.170952,53.200001,252.20,15.854602,25.31,28.52,2024-11-26


In [ ]:
getMarketData = ivol.setMethod('/equities/eod/earnings')
marketData = getMarketData(from_='2025-08-20', to='2030-10-10')
marketData

,load_date,symbol,company_name,market_cap,earning_date,time_of_day_code,estimate,stock_price,implied_move,straddle_call_contract,...,strangle_vol_call_ask,strangle_vol_put_ask,strangle_vol_probability_up,strangle_vol_probability_down,strangle_oi_call_contract,strangle_oi_put_contract,strangle_oi_call_ask,strangle_oi_put_ask,strangle_oi_probability_up,strangle_oi_probability_down
0,2025-09-24,COST,Costco Wholesale Corp,419245228032,2025-09-25,TAS,None,945.27,3.68,250926C00945000,...,3.05,1.31,0.0037,0.0016,250926C01000000,250926P00900000,3.05,2.87,0.0037,0.0247
1,2025-09-24,EWBC,East West Bancorp Inc,14575420416,2025-10-21,TAS,None,105.76,13.80,251121C00105000,...,3.80,2.20,0.1127,0.0000,251121C00120000,251121P00060000,2.50,2.20,0.0450,0.0000
2,2025-09-24,FITB,Fifth Third Bancorp,30160459776,2025-10-17,TAS,None,45.17,8.30,251121C00045000,...,1.40,0.85,0.2807,0.2379,251121C00048000,251121P00036000,0.70,0.45,0.0921,0.0000
3,2025-09-24,HBAN,Huntington Bancshares Inc/OH,25339357184,2025-10-17,TAS,None,17.37,9.79,251121C00017000,...,0.60,0.35,0.4917,0.5431,251121C00019000,251121P00016000,0.25,0.35,0.3017,0.5431
4,2025-09-24,RF,Regions Financial Corp,23819778048,2025-10-17,TAS,None,26.66,8.81,251121C00027000,...,0.65,0.55,0.2218,0.4704,251121C00028000,251121P00019000,0.65,0.55,0.2218,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2025-09-24,GLPG,Galapagos NV,2177236992,2025-10-23,TNS,None,33.04,30.27,251121C00032500,...,5.00,5.00,0.0799,0.0525,251121C00037500,251121P00020000,5.00,5.00,0.0799,0.0525
70,2025-09-24,NTR,Nutrien Ltd,28739639296,2025-11-06,TAS,None,59.03,9.99,251121C00060000,...,1.50,2.25,0.0001,0.5550,251121C00062500,251121P00052500,1.50,0.80,0.0001,0.0000
71,2025-09-24,SUZ,Suzano Papel e Celulose SA,11568107520,2025-11-07,TAS,None,9.36,16.56,251121C00010000,...,0.75,0.75,0.0088,0.0297,251121C00015000,251121P00005000,0.75,0.75,0.0088,0.0297
72,2025-09-24,DOW,Dow Inc,16395561984,2025-10-23,TAS,None,23.13,12.88,251024C00023000,...,0.42,0.61,0.0000,0.1021,251024C00030000,251024P00021000,0.19,0.61,0.0000,0.1021


In [ ]:
getMarketData = ivol.setMethod('/equities/eod/stock-opts-by-param')
marketData = getMarketData(symbol='AAPL',tradeDate='2021-12-16',dteFrom=0,dteTo=30,moneynessFrom=-10,moneynessTo=100,cp='C')
marketData

,c_date,option_symbol,dte,stocks_id,expiration_date,call_put,price_strike,price_open,price_high,price_low,...,gamma,theta,vega,rho,Ask,Bid,underlying_price,calc_OTM,option_id,is_settlement
0,2021-12-16,AAPL 211217C00157500,1,799,2021-12-17,C,157.5,22.51,23.57,13.60,...,0.008732,-0.318876,0.006725,0.004154,15.35,14.40,172.26,-8.57,118048511,0
1,2021-12-16,AAPL 211217C00160000,1,799,2021-12-17,C,160.0,19.09,21.03,11.00,...,0.012369,-0.354403,0.008439,0.004171,12.80,12.00,172.26,-7.12,115343545,0
2,2021-12-16,AAPL 211217C00162500,1,799,2021-12-17,C,162.5,16.22,18.52,8.76,...,0.015971,-0.289489,0.008665,0.004233,10.15,9.30,172.26,-5.67,118048513,0
3,2021-12-16,AAPL 211217C00165000,1,799,2021-12-17,C,165.0,14.19,16.05,6.35,...,0.029064,-0.458967,0.014722,0.004086,8.00,7.00,172.26,-4.21,115343547,0
4,2021-12-16,AAPL 211217C00167500,1,799,2021-12-17,C,167.5,11.38,13.55,4.25,...,0.049284,-0.517738,0.020363,0.003905,5.40,4.80,172.26,-2.76,118048515,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,2021-12-16,AAPL 220114C00225000,29,799,2022-01-14,C,225.0,0.26,0.26,0.17,...,0.002481,-0.021545,0.027027,0.003077,0.21,0.17,172.26,30.62,118159149,0
122,2021-12-16,AAPL 220114C00230000,29,799,2022-01-14,C,230.0,0.21,0.21,0.16,...,0.002097,-0.020092,0.023996,0.002669,0.18,0.16,172.26,33.52,118159151,0
123,2021-12-16,AAPL 220114C00235000,29,799,2022-01-14,C,235.0,0.12,0.12,0.12,...,0.001610,-0.015959,0.018738,0.001999,0.14,0.11,172.26,36.42,118159153,0
124,2021-12-16,AAPL 220114C00240000,29,799,2022-01-14,C,240.0,0.12,0.12,0.09,...,0.001330,-0.014076,0.015994,0.001663,0.12,0.06,172.26,39.32,118159155,0


In [38]:
getMarketData = ivol.setMethod('/equities/eod/ivs')
marketData = getMarketData(symbol='AAPL', date='2025-09-24')

marketData[marketData['period'] == 30]

,record_no,symbol,exchange,date,period,strike,out-of-the-money %,Call/Put,IV,delta
78,79,AAPL,NASDAQ,2025-09-24,30,252.310000,0,C,0.229867,0.534067
79,80,AAPL,NASDAQ,2025-09-24,30,252.310000,0,P,0.227618,-0.472269
80,81,AAPL,NASDAQ,2025-09-24,30,264.925480,5,C,0.220642,0.246691
81,82,AAPL,NASDAQ,2025-09-24,30,239.694490,5,P,0.252057,-0.215096
82,83,AAPL,NASDAQ,2025-09-24,30,277.541020,10,C,0.229191,0.086130
83,84,AAPL,NASDAQ,2025-09-24,30,227.079000,10,P,0.287696,-0.086961
84,85,AAPL,NASDAQ,2025-09-24,30,290.156500,15,C,0.247814,0.029854
85,86,AAPL,NASDAQ,2025-09-24,30,214.463500,15,P,0.336457,-0.038449
86,87,AAPL,NASDAQ,2025-09-24,30,302.772000,20,C,0.268131,0.011060
87,88,AAPL,NASDAQ,2025-09-24,30,201.848000,20,P,0.392458,-0.019213


In [ ]:
import os
import pandas as pd
import dotenv
import ivolatility as ivol

dotenv.load_dotenv()
API_KEY = os.getenv("IVOL_API_KEY")
if not API_KEY:
    raise RuntimeError("IVOL_API_KEY not found.")
ivol.setLoginParams(apiKey=API_KEY)

def third_friday(year: int, month: int) -> pd.Timestamp:
    start = pd.Timestamp(year=year, month=month, day=1)
    fridays = pd.date_range(start, start + pd.offsets.MonthEnd(0), freq="W-FRI")
    return fridays[2] if len(fridays) >= 3 else fridays[-1]

def month_bounds(year: int, month: int) -> tuple[pd.Timestamp, pd.Timestamp]:
    start = pd.Timestamp(year=year, month=month, day=1)
    end = start + pd.offsets.MonthEnd(0)
    return start.normalize(), end.normalize()

def choose_expiration(candidates: pd.Series, year: int, month: int) -> pd.Timestamp:
    ts = pd.to_datetime(candidates).dt.normalize()
    in_month = ts[(ts.dt.year == year) & (ts.dt.month == month)]
    if in_month.empty:
        return ts.iloc[(ts - third_friday(year, month)).abs().argsort().iloc[0]]
    target = third_friday(year, month).normalize()
    return in_month.iloc[(in_month - target).abs().argsort().iloc[0]]

def nearest_strike(strikes: pd.Series, target: float) -> dict:
    s = pd.Series(sorted(pd.to_numeric(strikes, errors="coerce").dropna().unique()))
    idx = (s - target).abs().argsort().iloc[0]
    below = s[s <= target].max() if not s[s <= target].empty else s.min()
    above = s[s >= target].min() if not s[s >= target].empty else s.max()
    return {"chosen": s.iloc[idx], "below": below, "above": above}

def fetch_chain(symbol: str, year: int, month: int, k_lo: float, k_hi: float):
    dt_start, dt_end = month_bounds(year, month)
    get_market = ivol.setMethod("/equities/option-series")

    calls = get_market(symbol=symbol, expFrom=dt_start.strftime("%Y-%m-%d"),
                       expTo=dt_end.strftime("%Y-%m-%d"),
                       strikeFrom=min(k_lo, k_hi)*0.5, strikeTo=max(k_lo, k_hi)*1.5,
                       callPut="C")
    puts  = get_market(symbol=symbol, expFrom=dt_start.strftime("%Y-%m-%d"),
                       expTo=dt_end.strftime("%Y-%m-%d"),
                       strikeFrom=min(k_lo, k_hi)*0.5, strikeTo=max(k_lo, k_hi)*1.5,
                       callPut="P")
    chain = pd.concat([calls, puts], ignore_index=True) if not calls.empty or not puts.empty else pd.DataFrame()
    return chain

def summarize_collar(symbol="AAPL", month="Oct", year=2025, put_k=200, call_k=300):
    output = {"ok": False, "issues": [], "normalized": {}, "files": {}}

    # 1) Normalize inputs (Validator)
    month_num = pd.to_datetime(f"01-{month}-{year}", dayfirst=True).month
    output["normalized"] = {"ticker": symbol.upper(),
                            "legs": [{"cp":"P","strike":float(put_k)},
                                     {"cp":"C","strike":float(call_k)}],
                            "month": month_num, "year": int(year)}

    # 2) Load chain for the month
    chain = fetch_chain(symbol.upper(), year, month_num, put_k, call_k)
    if chain.empty:
        output["issues"].append("No option data returned for month window.")
        return output

    # 3) Pick the expiration (prefer monthly 3rd Friday)
    maturity = choose_expiration(chain["expirationDate"], year, month_num)
    output["normalized"]["expiration"] = str(maturity.date())
    month_slice = chain[pd.to_datetime(chain["expirationDate"]).dt.normalize() == maturity]

    # 4) For each leg, pick exact/nearest strike and collect markets
    rows = []
    for cp, k_target in [("P", put_k), ("C", call_k)]:
        side = month_slice[month_slice["callPut"] == cp]
        if side.empty:
            output["issues"].append(f"No {cp} chain for {maturity.date()}.")
            continue
        picks = nearest_strike(side["strike"], k_target)
        chosen = side[side["strike"] == picks["chosen"]].copy()
        if chosen.empty:
            output["issues"].append(f"No strike near {k_target} for {cp}.")
            continue

        # Adjust these column names if your API uses different ones (e.g., 'bidPrice'/'askPrice','iv')
        for need in ["bid", "ask"]:
            if need not in chosen.columns:
                # fallback common alternatives
                alt = "bidPrice" if need=="bid" else "askPrice"
                if alt in chosen.columns:
                    chosen.rename(columns={alt: need}, inplace=True)

        chosen["mid"] = (pd.to_numeric(chosen["bid"], errors="coerce") + pd.to_numeric(chosen["ask"], errors="coerce")) / 2
        iv_col = "impliedVolatility" if "impliedVolatility" in chosen.columns else ("iv" if "iv" in chosen.columns else None)
        chosen["iv"] = pd.to_numeric(chosen[iv_col], errors="coerce") if iv_col else pd.NA

        row = chosen[["callPut","strike","expirationDate","bid","ask","mid","iv"]].iloc[0].to_dict()
        row["requested_strike"] = float(k_target)
        row["nearest_below"] = float(picks["below"])
        row["nearest_above"] = float(picks["above"])
        rows.append(row)

    if not rows:
        return output

    summary = pd.DataFrame(rows).sort_values(["callPut"])
    output["ok"] = True
    output["summary_table"] = summary.to_dict(orient="records")

    # Optional: persist CSV for UI “files” contract
    csv_path = f"./structure_eod_{symbol}_{year}{str(month_num).zfill(2)}.csv"
    summary.to_csv(csv_path, index=False)
    output["files"]["csv"] = csv_path

    # Value-only bullets (aligns to next step expectations)
    output["bullets"] = [
        f"Put {int(put_k)} bid/ask/mid: {summary.loc[summary.callPut=='P','bid'].iat[0]:.2f}/"
        f"{summary.loc[summary.callPut=='P','ask'].iat[0]:.2f}/"
        f"{summary.loc[summary.callPut=='P','mid'].iat[0]:.2f}",
        f"Call {int(call_k)} bid/ask/mid: {summary.loc[summary.callPut=='C','bid'].iat[0]:.2f}/"
        f"{summary.loc[summary.callPut=='C','ask'].iat[0]:.2f}/"
        f"{summary.loc[summary.callPut=='C','mid'].iat[0]:.2f}",
        f"Exp: {maturity.date()} | ATM IV? (next tool)"
    ]
    return output

# Example:
# result = summarize_collar(symbol="AAPL", month="Oct", year=2025, put_k=200, call_k=300)
# print(result["summary_table"]); print(result["files"])

In [2]:
result = summarize_collar(symbol="AAPL", month="Oct", year=2025, put_k=200, call_k=300)

KeyError: 'bid'

In [10]:
import os
import pandas as pd
import dotenv
import ivolatility as ivol

dotenv.load_dotenv()
API_KEY = os.getenv("IVOL_API_KEY")
if not API_KEY:
    raise RuntimeError("IVOL_API_KEY not found")
ivol.setLoginParams(apiKey=API_KEY)

# --- helpers ---------------------------------------------------------
def month_bounds(year: int, month: int):
    start = pd.Timestamp(year=year, month=month, day=1)
    end = start + pd.offsets.MonthEnd(0)
    return start.normalize(), end.normalize()

def third_friday(year: int, month: int) -> pd.Timestamp:
    start = pd.Timestamp(year=year, month=month, day=1)
    fridays = pd.date_range(start, start + pd.offsets.MonthEnd(0), freq="W-FRI")
    return fridays[2] if len(fridays) >= 3 else fridays[-1]

def choose_expiration(dates: pd.Series, year: int, month: int) -> pd.Timestamp:
    s = pd.to_datetime(dates).dt.normalize()
    target = third_friday(year, month).normalize()
    in_month = s[(s.dt.year == year) & (s.dt.month == month)]
    base = in_month if not in_month.empty else s
    return base.iloc[(base - target).abs().argsort().iloc[0]]

def nearest_strike(series: pd.Series, target: float) -> dict:
    s = pd.Series(sorted(pd.to_numeric(series, errors="coerce").dropna().unique()))
    if s.empty:
        return {"chosen": None, "below": None, "above": None, "exact": False}
    idx = (s - target).abs().argsort().iloc[0]
    below = s[s <= target].max() if not s[s <= target].empty else s.min()
    above = s[s >= target].min() if not s[s >= target].empty else s.max()
    return {"chosen": float(s.iloc[idx]), "below": float(below), "above": float(above), "exact": float(s.iloc[idx]) == float(target)}

# --- core ------------------------------------------------------------
def build_collar_skeleton(symbol="AAPL", year=2025, month=10, put_k=200, call_k=300):
    output = {"ok": False, "issues": [], "normalized": {}, "summary": []}

    # Normalize
    symbol = symbol.upper()
    dt_start, dt_end = month_bounds(year, month)
    output["normalized"].update({
        "ticker": symbol, "year": int(year), "month": int(month),
        "legs": [{"cp":"P","strike":float(put_k)}, {"cp":"C","strike":float(call_k)}]
    })

    # Fetch series (no quotes here)
    get_series = ivol.setMethod('/equities/option-series')
    calls = get_series(symbol=symbol,
                       expFrom=dt_start.strftime('%Y-%m-%d'),
                       expTo=dt_end.strftime('%Y-%m-%d'),
                       callPut='C')
    puts  = get_series(symbol=symbol,
                       expFrom=dt_start.strftime('%Y-%m-%d'),
                       expTo=dt_end.strftime('%Y-%m-%d'),
                       callPut='P')
    if (calls.empty and puts.empty):
        output["issues"].append("No series data returned for the month window.")
        return output

    chain = pd.concat([calls, puts], ignore_index=True)

    # Pick a single expiry (prefer 3rd Friday in the requested month)
    maturity = choose_expiration(chain["expirationDate"], year, month)
    output["normalized"]["expiration"] = str(maturity.date())
    month_slice = chain[pd.to_datetime(chain["expirationDate"]).dt.normalize() == maturity]

    # Build skeleton rows for each leg
    rows = []
    for cp, k in [("P", put_k), ("C", call_k)]:
        side = month_slice[month_slice["callPut"] == cp]
        if side.empty:
            output["issues"].append(f"No {cp} series for {maturity.date()}.")
            continue

        pick = nearest_strike(side["strike"], float(k))
        if pick["chosen"] is None:
            output["issues"].append(f"No strikes found near {k} for {cp}.")
            continue

        chosen = side[side["strike"] == pick["chosen"]].copy()
        # Try to map a likely IV column if present; otherwise leave None
        iv_col = None
        for candidate in ["impliedVolatility", "iv", "impVol", "imVol"]:
            if candidate in chosen.columns:
                iv_col = candidate
                break

        row = {
            "leg": "Long Put" if cp == "P" else "Short Call",   # change as needed
            "cp": cp,
            "requested_strike": float(k),
            "matched_strike": float(pick["chosen"]),
            "nearest_below": pick["below"],
            "nearest_above": pick["above"],
            "exact_match": pick["exact"],
            "expiration": str(maturity.date()),
            # placeholders; a later step will fill these from a quotes endpoint
            "bid": None,
            "ask": None,
            "iv": float(chosen[iv_col].iloc[0]) if iv_col else None,
            "note": "quotes_pending"
        }
        rows.append(row)

    if not rows:
        return output

    summary = pd.DataFrame(rows).sort_values(["cp"])
    output["ok"] = True
    output["summary"] = summary.to_dict(orient="records")
    return output

# Example:
# result = build_collar_skeleton(symbol="AAPL", year=2025, month=10, put_k=200, call_k=300)
# result

In [11]:
build_collar_skeleton(symbol="AAPL", year=2025, month=10, put_k=200, call_k=300)


{'ok': True,
 'issues': [],
 'normalized': {'ticker': 'AAPL',
  'year': 2025,
  'month': 10,
  'legs': [{'cp': 'P', 'strike': 200.0}, {'cp': 'C', 'strike': 300.0}],
  'expiration': '2025-10-17'},
 'summary': [{'leg': 'Short Call',
   'cp': 'C',
   'requested_strike': 300.0,
   'matched_strike': 300.0,
   'nearest_below': 300.0,
   'nearest_above': 300.0,
   'exact_match': True,
   'expiration': '2025-10-17',
   'bid': None,
   'ask': None,
   'iv': None,
   'note': 'quotes_pending'},
  {'leg': 'Long Put',
   'cp': 'P',
   'requested_strike': 200.0,
   'matched_strike': 200.0,
   'nearest_below': 200.0,
   'nearest_above': 200.0,
   'exact_match': True,
   'expiration': '2025-10-17',
   'bid': None,
   'ask': None,
   'iv': None,
   'note': 'quotes_pending'}]}

In [ ]:
from polygon import RESTClient

client = RESTClient("iASuja66NhcDdq_yQu7mhYZMzBqQdA2O")

dividends = []
for d in client.list_dividends(
	order="asc",
	limit=10,
	sort="ex_dividend_date",
	):
    dividends.append(d)

print(dividends)